In [26]:
import pymongo
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["story_database"]  # Database name
collection = db["short_stories"]

documents = collection.find()

df_raw = pd.DataFrame(list(documents))
df_raw = df_raw[["text", "label"]]
df_raw.head(5)

,text,label
0,Transcriber's Note:\nEvery effort has been mad...,0
1,"“In a few moments Marianne, Solomin, Paul,\n\n...",0
2,“Marianne knelt beside the sofa.… Nezhdanof\n\...,0
3,"“Solomin raised Marianne's hand, her head\n\nl...",0
4,"“He now was no longer, but the hands of\n\nSol...",0


In [27]:
# Define the cleaning function
def clean_text(text):

    text = text.replace('\n', ' ') 
    text = re.sub(r'=', '', text)  

    # Keep only English alphabet characters and spaces 
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

df_raw['cleaned_text'] = df_raw['text'].apply(clean_text)

df = df_raw[["cleaned_text", "label"]]

# Vectorize the data


In [28]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df_shuffled = df.sample(frac=1).reset_index(drop=True)


X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

cleaned_data = (classification_report(y_test, y_pred))
print(cleaned_data)

              precision    recall  f1-score   support

           0       1.00      0.50      0.67        44
           1       0.66      1.00      0.80        43

    accuracy                           0.75        87
   macro avg       0.83      0.75      0.73        87
weighted avg       0.83      0.75      0.73        87



Wow this was a bit anticlimactic, lets mess the data up see if we get the same results

# Lets not clean the data!

In [29]:
df_shuffled = df_raw.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.75      0.86        40
           1       0.82      1.00      0.90        47

    accuracy                           0.89        87
   macro avg       0.91      0.88      0.88        87
weighted avg       0.91      0.89      0.88        87



# Lets add data, fakenews!

In [30]:
fakenews = pd.read_csv("data/fake_news.csv")
fakenews.columns = ["cleaned_text", "label"]

In [31]:
data = pd.concat([fakenews, df], ignore_index=True)


In [32]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.82      0.90        44
           1       0.73      0.98      0.83        44
           2       0.93      0.80      0.86        49

    accuracy                           0.86       137
   macro avg       0.89      0.86      0.86       137
weighted avg       0.89      0.86      0.86       137



In [33]:
confusion_matrix(y_test, y_pred)

array([[36,  6,  2],
       [ 0, 43,  1],
       [ 0, 10, 39]], dtype=int64)

## Is the vectorizer that good?

In [34]:
bbc_news = pd.read_csv("data/bbc_news.csv")
bbc = bbc_news[["cleaned_text", "category_encoded"]]
bbc.columns =  ["cleaned_text", "label"]

In [35]:
data = pd.concat([bbc, data], ignore_index=True)

In [36]:
data["label"].value_counts()

label
3    100
6    100
5    100
7    100
4    100
2    100
1    100
0     74
Name: count, dtype: int64

In [37]:
df_shuffled = data.sample(frac=1).reset_index(drop=True)

X = df_shuffled["cleaned_text"]
y = df_shuffled["label"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


clf = LogisticRegression(max_iter=1000)  # max_iter to ensure convergence
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98        21
           1       0.74      0.93      0.82        27
           2       0.72      0.75      0.73        24
           3       0.91      0.75      0.82        28
           4       0.94      0.94      0.94        34
           5       0.95      1.00      0.97        37
           6       0.97      0.97      0.97        32
           7       1.00      0.87      0.93        30

    accuracy                           0.90       233
   macro avg       0.90      0.89      0.90       233
weighted avg       0.91      0.90      0.90       233



In [38]:
confusion_matrix(y_test, y_pred)

array([[20,  0,  1,  0,  0,  0,  0,  0],
       [ 0, 25,  2,  0,  0,  0,  0,  0],
       [ 0,  5, 18,  0,  1,  0,  0,  0],
       [ 0,  1,  3, 21,  1,  2,  0,  0],
       [ 0,  1,  1,  0, 32,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 37,  0,  0],
       [ 0,  0,  0,  1,  0,  0, 31,  0],
       [ 0,  2,  0,  1,  0,  0,  1, 26]], dtype=int64)

## Increasing the datas dimensions by adding more data as well as adding more things to classify is instead of worsening the model its getting improved?
The TfidfVectorizer proves to be an excellent tool for vectorizing text data, as it does a near-perfect job of capturing the essence of the information. However, it's important to note that a vectorizer like TfidfVectorizer thrives on large datasets. While adding more labels was part of the improvement, it was the accompanying increase in data that enabled the creation of a more complex sparse matrix, allowing the model to perform more efficiently and effectively.

## Lets create a transformer

In [39]:
df_deeplearning = df_shuffled

In [61]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Use tokenizer to encode
df_deeplearning['input_ids'] = df_deeplearning['cleaned_text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)

In [43]:
df_deeplearning

,cleaned_text,label,input_ids
0,Mobiles get set for visual radio The growth i...,4,"[101, 4684, 2015, 2131, 2275, 2005, 5107, 2557..."
1,Controlled designation of origin may refer to ...,1,"[101, 4758, 8259, 1997, 4761, 2089, 6523, 2000..."
2,Drink remark acts as diversion The first mini...,5,"[101, 4392, 17674, 4490, 2004, 20150, 1996, 20..."
3,Car giant hit by Mercedes slump A slump in pr...,3,"[101, 2482, 5016, 2718, 2011, 10793, 28702, 10..."
4,Neeson in bid to revive theatre Hollywood fil...,6,"[101, 7663, 3385, 1999, 7226, 2000, 17995, 300..."
...,...,...,...
769,Asian quake hits European shares Shares in Eu...,3,"[101, 4004, 27785, 4978, 2647, 6661, 6661, 199..."
770,The Sound of Music is coming home The origina...,6,"[101, 1996, 2614, 1997, 2189, 2003, 2746, 2188..."
771,Man City 0-2 Man Utd Manchester United reduce...,7,"[101, 2158, 2103, 1014, 1011, 1016, 2158, 2118..."
772,Apple unveils low-cost Mac mini Apple has unv...,4,"[101, 6207, 4895, 3726, 12146, 2659, 1011, 346..."


In [87]:
input_ids = torch.tensor(df_deeplearning['input_ids'].tolist(), dtype=torch.long)
labels = torch.tensor(df_deeplearning['label'].tolist(), dtype=torch.float)  # Assuming multi-label

# Create torch dataset
dataset = TensorDataset(input_ids, labels)

# Create a DataLoader for batching and shuffling
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [104]:
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_units, output_size):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_units))
        self.layer2 = nn.Linear(hidden_units, output_size)


    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)  
        return x
    

model_0 = SimpleModel(input_size=128, hidden_units=16, output_size=8)

In [105]:
input_ids.size()

torch.Size([774, 128])

In [106]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(), lr = 0.01)

In [107]:
def training_loop(loss_fn, optimizer, model,epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        train_loss = 0
        for batch, (X, y) in enumerate(dataloader):
            model.train()
            

            X, y = X.to(device), y.to(device)

            X = X.float() 
            y = y.long()  
            
            y_pred = model_0(X)

            loss = loss_fn(y_pred, y)
            train_loss += loss.item()

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

        
        train_loss /= len(dataloader)
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {train_loss:.4f}")


# Call the training loop
training_loop(loss_fn=loss_fn, optimizer=optimizer, model=model_0, epochs=100)


Epoch 1/100, Average Training Loss: 99.9596
Epoch 21/100, Average Training Loss: 2.0679
Epoch 41/100, Average Training Loss: 2.0677
Epoch 61/100, Average Training Loss: 2.0693
Epoch 81/100, Average Training Loss: 2.0691
